In [ ]:
# import some common libraries
import os
import numpy as np
import cv2
import random
import pandas as pd
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.engine import DefaultTrainer

# Root directory of the project
ROOT_DIR = os.path.join("..\Computer_Vision_Eindopdracht\deep_learning") # Go one folder up

#dataset 
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, os.path.join(ROOT_DIR, "train", "_annotations.coco.json"),os.path.join(ROOT_DIR,'train'))
register_coco_instances("my_dataset_val", {}, os.path.join(ROOT_DIR,"valid", "_annotations.coco.json"),os.path.join(ROOT_DIR,'valid'))

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val")
cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
import pandas as pd
df = pd.read_csv('deep_learning/output_data_det.csv')
df


In [ ]:
from detectron2.utils.visualizer import ColorMode
from custom_config_detectron2 import *

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold

In [ ]:
from matplotlib import pyplot as plt
from math import sqrt

white: np.ndarray  = np.array(["white",  (210, 210, 210)],   dtype=object)
black: np.ndarray  = np.array(["black",  (30,  30,  30 )],    dtype=object)
metal: np.ndarray  = np.array(["metal",  (120, 120, 120)], dtype=object)

pink:  np.ndarray  = np.array(["pink",   (120, 90,  220)], dtype=object)
blue:  np.ndarray  = np.array(["blue",   (255, 100 ,10 )],  dtype=object)

colors: list = (white, black, metal)

lower_background_color = (13,     145,      20)
upper_background_color = (35,     255,     255)
lower_blue             = (30,     120,      20)
upper_blue             = (120,    255,     255)
lower_pink             = (130,    50,      20)
upper_pink             = (180,    255,     255)

im = cv2.imread("deep_learning/test/Image_Yellow (201).jpg")
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

imgHLS = cv2.cvtColor(im, cv2.COLOR_BGR2HLS)
hsv_image = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

mask = outputs['instances'].pred_masks.numpy()

mask5 = cv2.inRange(hsv_image, lower_background_color, upper_background_color) #============================================
mask_binary = cv2.bitwise_not(mask5) #============================================
mask_with_color = cv2.bitwise_and(hsv_image, im, mask = mask_binary) #============================================

contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

mask = mask.astype(np.uint8)
mask[mask > 0] = 255

fontFace = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
borderThickness = 2
index = 1


for i in range(mask.shape[0]):
    # Creating threshold of the manually created mask and then finding contours of roadmarks
    gray = mask[i,:,:]
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    contour = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    cntrs = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contour, key=cv2.contourArea)

    mask_binary = cv2.bitwise_not(mask5)
    mask_with_color = cv2.bitwise_and(hsv_image, im, mask = mask_binary)
    contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for contour in contours:
        maskr = np.zeros(im.shape[:2], dtype="uint8")
        cv2.drawContours(maskr, [contour], -1, 255, -1)

        # check pink
        pink_mask = np.zeros(result.shape[:2], dtype="uint8")
        cv2.drawContours(pink_mask, [contour], -1, 255, -1)
        individual_masks = cv2.bitwise_and(result, result, mask = pink_mask)
        pink_mask = cv2.inRange(individual_masks, lower_pink, upper_pink)

        if np.mean(pink_mask) > 0:
            match_color = 'pink'

        mean = cv2.mean(im, mask=maskr)[0:3]
        min_rmse = 1000000
        # print(mean)
        for color in colors:
            bb = color[1][0]
            gg = color[1][1]
            rr = color[1][2]
            rmse = sqrt( ( (mean[2]-rr)*(mean[2]-rr) + (mean[1]-gg)*(mean[1]-gg) + (mean[0]-bb)*(mean[0]-bb) )/3 )

            if rmse < min_rmse:
                min_rmse = rmse
                match_color = color[0]
    #qprint(match_color)

    # Create three channel mask for single part of the roadmark
    mask_part = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # Count every pixel in mask
    pixels = np.sum(gray > 0)

    # Combine image and mask to remove noise
    result = cv2.bitwise_and(imgHLS, mask_part)
    
    # cv2.namedWindow("Resized_Window", cv2.WINDOW_KEEPRATIO)
    # cv2.imshow("Resized_Window", result)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
        
# cv2.imwrite("deep_learning/results/ouput_img.jpg",out.get_image()[:, :, ::-1])
# cv2.namedWindow("Resized_Window", cv2.WINDOW_KEEPRATIO)
# cv2.imshow("Resized_Window", mask[1:])
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# import the modules
import glob

# get the path/directory
TEST_DIR = "deep_learning/test"
RES_DIR = "deep_learning/results"
# Counter var for filenames
d = 0

# Using glob to loop over image dir
for images in glob.iglob(f'{TEST_DIR}/*'):
    # Collecting all images of type '.jpg'
    if images.endswith(".jpg"):
        # Masking images
        im = cv2.imread(images)
        outputs = predictor(im)
        v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        # Saving files
        filename = "deep_learning/results_2000/classified_img_%d.jpg"%d
        cv2.imwrite(filename, out.get_image()[:, :, ::-1])
        d+=1


In [ ]:
im = cv2.imread("deep_learning/test/Image_Yellow (201).jpg")
imgHLS = cv2.cvtColor(im, cv2.COLOR_BGR2HLS)

outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2.imshow('mask', out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()
mask_array = outputs['instances'].pred_masks.numpy()
classes = outputs['instances'].pred_classes.numpy()
#orientation = outputs['instances'].pred_orientation.numpy()
print(classes)

mask = mask_array.astype(np.uint8)
mask[mask > 0] = 255



for i in range(mask.shape[0]):
    # Creating threshold of the manually created mask and then finding contours of roadmarks
    gray = mask[i,:,:]
    thresh = cv2.threshold(
        gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    contour = cv2.findContours(
        thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    contour = max(contour, key=cv2.contourArea)

    # Create three channel mask for single part of the roadmark
    mask_part = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

    # Count every pixel in mask
    pixels = np.sum(gray > 0)
    

result = cv2.bitwise_and(imgHLS, mask)

In [ ]:
classes = outputs['instances'].pred_classes
scores = outputs['instances'].scores
keypoints = outputs['instances'].pred_boxes
classes

# CLASSES

In [ ]:
a = classes.cpu().detach().numpy()
a = str(a)
a = a.replace('0', 'bolt')
a = a.replace('1', 'nut')
a = a.replace('2', 'ring')
a = a.replace('3', 'metal attachment')
a = a.replace('4', 'valve')

    
a


# LOCATION

In [ ]:
keypoints = outputs['instances'].pred_boxes.tensor.numpy()
keypoints
# centerx,centery = ( np.average(keypoints[:2]),np.average(keypoints[2:]))


# ORIENTATION

In [ ]:
(_,_),(_,_),angle = cv.fitEllipse(contour)

# COLOR


In [ ]:
alist = outputs['instances'].pred_masks.numpy()
alist = alist[0]
alist
mask = alist.astype(np.uint8)
mask[mask > 0] = 255
mask 


In [ ]:
G

In [ ]:
from math import sqrt

white: np.ndarray  = np.array(["white",  (210, 210, 210)],   dtype=object)
black: np.ndarray  = np.array(["black",  (30,  30,  30 )],    dtype=object)
metal: np.ndarray  = np.array(["metal",  (120, 120, 120)], dtype=object)

pink:  np.ndarray  = np.array(["pink",   (120, 90,  220)], dtype=object)
blue:  np.ndarray  = np.array(["blue",   (255, 100 ,10 )],  dtype=object)

colors: list = (white, black, metal)



    
mask = np.zeros(original_image.shape[:2], dtype="uint8")
cv2.drawContours(mask, [contour], -1, 255, -1)

# check pink
pink_mask = np.zeros(cont_mask.shape[:2], dtype="uint8")
cv2.drawContours(pink_mask, [contour], -1, 255, -1)
individual_masks = cv2.bitwise_and(cont_mask, cont_mask, mask = pink_mask)
pink_mask = cv2.inRange(individual_masks, lower_pink, upper_pink)

if np.mean(pink_mask) > 0:
    return 'pink'

mean = cv2.mean(original_image, mask=mask)[0:3]
min_rmse = 1000000
# print(mean)
for color in colors:
    bb = color[1][0]
    gg = color[1][1]
    rr = color[1][2]
    rmse = sqrt( ( (mean[2]-rr)*(mean[2]-rr) + (mean[1]-gg)*(mean[1]-gg) + (mean[0]-bb)*(mean[0]-bb) )/3 )

    if rmse < min_rmse:
        min_rmse = rmse
        match_color = color[0]
    





# CSV

In [ ]:
import csv

with open('./output_data_det.csv', 'a', encoding='UTF8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    for contour in range(len(img.contours)):
         csv_writer.writerow((img.name, img.contours[contour].kind_of_object, 1, img.contours[contour].position, img.contours[contour].oriëntation, img.contours[contour].color))


%load_ext tensorboard

%load_ext tensorboard
%tensorboard --logdir output

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output_5000